# ETL PROCCES USER ITEMS

In [35]:
import json
import ast
import re
import gzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns



In [36]:
def load_json_lines(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            data.append(ast.literal_eval(line))
    return pd.DataFrame(data)
data_items = load_json_lines("./users_items.json/australian_users_items.json")

In [37]:
data_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


## Proceso de analisis:
En el proceso de extracción y limpieza de datos, primero desanidaremos los datos de la columna ‘items’, creando nuevas columnas para ‘item_id’ y ‘item_name’. Luego, limpiaremos los datos eliminando espacios en blanco innecesarios y asegurándonos de que todos los ‘item_name’ estén en minúsculas para la consistencia. También transformaremos los ‘item_id’ a números enteros si es necesario. Manejaremos los valores faltantes, ya sea eliminando las filas correspondientes o rellenándolos con un valor predeterminado. Finalmente comprobaremos que nuestro proceso de ETL sea exitoso. 

In [38]:
data_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [39]:
data_items.shape

(88310, 5)

In [40]:
type(data_items)

pandas.core.frame.DataFrame

## Proceso De transformaición

Primero vamos a desanidar los datos según lo sugiere el dataset

In [41]:
# Convierte el DataFrame a una lista de diccionarios
data_items_dict = data_items.to_dict('records')

# Ahora puedes usar pd.json_normalize()
data_items_desanidado = pd.json_normalize(data_items_dict, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'])



In [10]:
data_items_desanidado

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [42]:
# hacemos un analisis para todo el df con tal de ver la clase de cada datos de columnas 
tipo_data = {"columna":[],"tipos_de_datos":[]} #genero un diccionario vacio para ir almacenando lo que genere el bucle

for columna in data_items_desanidado.columns: #un bucle que va recorriendo 
    tipo_data["columna"].append(columna)
    tipo_data["tipos_de_datos"].append(data_items_desanidado[columna].apply(type).unique())

analisis_df= pd.DataFrame(tipo_data)
analisis_df

,columna,tipos_de_datos
0,item_id,[<class 'str'>]
1,item_name,[<class 'str'>]
2,playtime_forever,[<class 'int'>]
3,playtime_2weeks,[<class 'int'>]
4,steam_id,[<class 'str'>]
5,items_count,[<class 'int'>]
6,user_id,[<class 'str'>]
7,user_url,[<class 'str'>]


In [43]:
data_items_desanidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   steam_id          object
 5   items_count       object
 6   user_id           object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


In [13]:
data_items_desanidado.head(3)

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [44]:
# Contar los datos nulos en cada columna
nulos = data_items_desanidado.isnull().sum()
print("Datos nulos en cada columna:")
nulos

Datos nulos en cada columna:


item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
steam_id            0
items_count         0
user_id             0
user_url            0
dtype: int64

In [45]:
for columna in data_items_desanidado.columns:
    duplicados_columna = data_items_desanidado[columna].duplicated().sum()
    print(f"\nDatos duplicados en la columna {columna}:")
    print(duplicados_columna)


Datos duplicados en la columna item_id:
5142231

Datos duplicados en la columna item_name:
5142262

Datos duplicados en la columna playtime_forever:
5104348

Datos duplicados en la columna playtime_2weeks:
5148759

Datos duplicados en la columna steam_id:
5082297

Datos duplicados en la columna items_count:
5152285

Datos duplicados en la columna user_id:
5082297

Datos duplicados en la columna user_url:
5082297


In [46]:
# Cantidad total de datos en el DataFrame
total_datos = data_items_desanidado.size
print("Cantidad total de datos en el DataFrame:")
print(total_datos)

Cantidad total de datos en el DataFrame:
41225672


## proceso de limpieza 

El proceso de limpieza de datos es un paso crucial en cualquier análisis de datos. Asegurarse de que los datos estén limpios y en el formato correcto puede ayudar a obtener resultados más precisos y significativos. A continuación, se describe el proceso de limpieza de datos que se utilizó para el DataFrame data_items_desanidado:

In [47]:
duplicados_comparativa= data_items_desanidado.loc[data_items_desanidado.duplicated()]
duplicados_comparativa

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
164294,20,Team Fortress Classic,5,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164295,50,Half-Life: Opposing Force,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164296,70,Half-Life,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164297,130,Half-Life: Blue Shift,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164298,220,Half-Life 2,198,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
...,...,...,...,...,...,...,...,...
4898223,213670,South Park™: The Stick of Truth™,725,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898224,221910,The Stanley Parable,53,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898225,261030,The Walking Dead: Season Two,253,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898226,273110,Counter-Strike Nexon: Zombies,0,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...


In [18]:
data_items_desanidado = data_items_desanidado.drop_duplicates(keep='first')
data_items_desanidado

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [48]:
data_items_desanidado = data_items_desanidado.copy()

In [49]:
# Calculamos cada columna del DataFrame con el fin de verificar que esté limpia
for col in data_items_desanidado.columns:
    print(f"Columna: {col}")

    # Imprimimos la frecuencia de los valores en cada columna
    print(data_items_desanidado[col].value_counts())

    # Contamos los valores faltantes en cada columna
    print(f"Valores faltantes: {data_items_desanidado[col].isnull().sum()}")

Columna: item_id
item_id
205790    49571
730       43776
4000      43301
304930    38682
223530    37044
          ...  
495720        1
526240        1
451010        1
465150        1
485270        1
Name: count, Length: 10978, dtype: int64
Valores faltantes: 0
Columna: item_name
item_name
Dota 2 Test                            49571
Counter-Strike: Global Offensive       43776
Garry's Mod                            43301
Unturned                               38682
Left 4 Dead 2                          37044
                                       ...  
My Own Pet                                 1
Medieval Battlefields Black Edition        1
Football VR                                1
Tower Dwellers                             1
Arachnophobia                              1
Name: count, Length: 10947, dtype: int64
Valores faltantes: 0
Columna: playtime_forever
playtime_forever
0         1867963
1          102730
2           34770
3           31905
4           29438
           ...   


In [50]:


# Verificar cada nombre de columna
for col in data_items_desanidado.columns:
    contains_special_chars = any(char not in valid_chars for char in col)
    print(f"¿La columna '{col}' contiene caracteres especiales? {contains_special_chars}")

¿La columna 'item_id' contiene caracteres especiales? False
¿La columna 'item_name' contiene caracteres especiales? False
¿La columna 'playtime_forever' contiene caracteres especiales? False
¿La columna 'playtime_2weeks' contiene caracteres especiales? False
¿La columna 'steam_id' contiene caracteres especiales? False
¿La columna 'items_count' contiene caracteres especiales? False
¿La columna 'user_id' contiene caracteres especiales? False
¿La columna 'user_url' contiene caracteres especiales? False


: 

In [21]:
def verificar_limpieza(df):
    # Verificar si hay datos duplicados
    duplicados = df.duplicated().any()
    
    # Verificar si hay datos nulos
    nulos = df.isnull().any().any()
    
    # Si no hay datos duplicados ni nulos, la limpieza es óptima
    if not duplicados and not nulos:
        print("La limpieza del DataFrame es óptima.")
    else:
        print("La limpieza del DataFrame no es óptima.")
        if duplicados:
            print("Existen datos duplicados en el DataFrame.")
        if nulos:
            print("Existen datos nulos en el DataFrame.")

# Usar la función para verificar la limpieza de data_items_desanidado
verificar_limpieza(data_items_desanidado)

La limpieza del DataFrame es óptima.


In [27]:
# Guardamos en formato CSV
data_items_desanidado.to_csv('data_items_desanidado_cleaned.csv', index=False)

# Guardar en formato json
data_items_desanidado.to_json('data_items_desanidado_cleaned.json', orient='records', lines=True)



In [31]:
import pyarrow
import fastparquet
data_items_desanidado.to_parquet('user_items_cleaned.parquet', index=False)